In [1]:
import importlib
import datamanip
importlib.reload(datamanip)

<module 'datamanip' (namespace) from ['C:\\Users\\pavel\\PycharmProjects\\ADES-reliability\\src\\datamanip']>

In [2]:
import torch
from filepath import *
from datamanip.plots import generate_matrix, generate_metrics
from models.GAT import GAT#, GCN, SimpleMPNN
from train_eval.train import train
from train_eval.evaluate import evaluate
from datamanip.datasetmanip.three_five_dataset import ThreeFiveDataset
from datamanip.datasetmanip.dataset_util import create_loaders

# Model Selection by number of 9's
### 1. Match 1-1: 0 vs. 7
### 2. Match 2-1: 0/1 vs. 2/3
### 3. Match 2-2: 4/5 vs. 6/7
### 4. Match 3-1: 0 vs. 1
### 5. Match 3-2: 2 vs. 3
### 6. Match 3-3: 4 vs. 5
### 7. Match 3-4: 6 vs. 7



In [3]:
match = "2-1"

In [4]:
if match == "1-1":
    NUM_EPOCHS = 200
    DROPOUT_RATE = 0.15
    LEARNING_RATE = 0.0008
    NODE_FEATURES = 12
    PATIENCE = 30
    HIDDEN_DIM = 128
elif match == "2-1":
    NUM_EPOCHS = 200
    DROPOUT_RATE = 0.15
    LEARNING_RATE = 0.0008
    NODE_FEATURES = 12
    PATIENCE = 15
    HIDDEN_DIM = 16
elif match == "2-2":
    NUM_EPOCHS = 150
    DROPOUT_RATE = 0.3
    LEARNING_RATE = 0.0005
    NODE_FEATURES = 12
    PATIENCE = 15
    HIDDEN_DIM = 28
elif match == "3-1":
    NUM_EPOCHS = 150
    DROPOUT_RATE = 0.3
    LEARNING_RATE = 0.0005
    NODE_FEATURES = 12
    PATIENCE = 15
    HIDDEN_DIM = 82
elif match == "3-2":
    NUM_EPOCHS = 200
    DROPOUT_RATE = 0.3
    LEARNING_RATE = 0.0001
    NODE_FEATURES = 12
    PATIENCE = 30
    HIDDEN_DIM = 128
elif match == "3-3":
    NUM_EPOCHS = 200
    DROPOUT_RATE = 0.3
    LEARNING_RATE = 0.0005
    NODE_FEATURES = 12
    PATIENCE = 30
    HIDDEN_DIM = 64
elif match == "3-4":
    NUM_EPOCHS = 200
    DROPOUT_RATE = 0.3
    LEARNING_RATE = 0.0005
    NODE_FEATURES = 12
    PATIENCE = 20
    HIDDEN_DIM = 32
else:
    NUM_EPOCHS = 100
    DROPOUT_RATE = 0.2
    LEARNING_RATE = 0.0007
    NODE_FEATURES = 12
    PATIENCE = 15
    HIDDEN_DIM = 64

In [5]:
#Create list of Data objects, each containing the node features, edge indices, and target values
train_list = ThreeFiveDataset(root=dataset_path, match=match, test_train_val="train")
test_list = ThreeFiveDataset(root=dataset_path, match=match, test_train_val="test")
val_list = ThreeFiveDataset(root=dataset_path, match=match, test_train_val="val")

Processing...


Finished splitting dataset


C:\Users\pavel\PycharmProjects\ADES-reliability\src\datamanip\dataset_parts_construction.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['matrix'] = df['matrix'].apply(np.array)
53793it [07:30, 119.52it/s]


torch.Size([53793, 26, 12])
Finished processing 1 out of 7 ranges
Finished processing ensemble dataset


17932it [02:29, 120.10it/s]


...

2783it [00:23, 119.83it/s]


torch.Size([2783, 26, 12])
Finished processing 7 out of 7 ranges


2834it [00:23, 120.31it/s]


torch.Size([2834, 26, 12])
Finished processing 7 out of 7 ranges


Done!


In [6]:
train_loader, val_loader, test_loader = create_loaders(train_list, val_list, test_list)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GAT(input_dim=NODE_FEATURES, hidden_dim=HIDDEN_DIM, output_dim=2, dropout_rate=DROPOUT_RATE).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = torch.nn.CrossEntropyLoss()

In [8]:
train_config = {
    "num_epochs": NUM_EPOCHS,
    "patience": PATIENCE,
    "device": device,
    "criterion": criterion,
    "optimizer": optimizer,
}

In [9]:
train(model, train_loader, val_loader, train_config, model_checkpoints_path + "/booster/" + match + ".pth")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (416x12 and 10x16)

In [ ]:
# Load the best model after training
model.load_state_dict(torch.load(model_checkpoints_path + "/booster/" + match + ".pth", weights_only=True))

In [ ]:
true_values, predicted_values, accuracy = evaluate(device, model, test_loader, model_checkpoints_path + "/booster/" + match + ".pth")

In [ ]:
generate_matrix(true_values, predicted_values, accuracy, "GAT Match: "  + match)

In [ ]:
generate_metrics(true_values, predicted_values, match, model_checkpoints_path, NUM_EPOCHS, LEARNING_RATE, NODE_FEATURES, DROPOUT_RATE, PATIENCE, HIDDEN_DIM)